# Creating a Sentiment Analysis Web App
## Using PyTorch and SageMaker


## General Outline


1. Download or otherwise retrieve the data.
2. Process / Prepare the data.
3. Upload the processed data to S3.
4. Train a chosen model.
5. Test the trained model (typically using a batch transform job).
6. Deploy the trained model.
7. Use the deployed model.

## Step 1: Downloading the data

In this project, I will be using the [IMDb dataset](http://ai.stanford.edu/~amaas/data/sentiment/)


In [1]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

--2020-08-09 14:08:36--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘../data/aclImdb_v1.tar.gz’

../data/aclImdb_v1. 100%[===================>]  80.23M  24.4MB/s    in 4.0s    

2020-08-09 14:08:40 (19.8 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]



## Step 2: Preparing and Processing the data

Read in each of the reviews and combine them into a single input structure. Then, split the dataset into a training set and a testing set.

In [2]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [3]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


In [4]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [5]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 25000, test = 25000


In [6]:
# Print example of the data our model will be trained on
print(train_X[100])
print(train_y[100])

i can't believe that NONE of the official reviews for this movie warn people that it contains two quite upsetting sexual assault scenes. It's as though our culture accepts this kind of behavior as simply sexual but not violent. My biggest problem with the movie is that it doesn't seem to condemn these assaults - as in, the woman who is repeatedly assaulted and pressured never holds the men accountable for their actions, and neither does anyone else. One man is stopped from completing the assault when someone throws a dagger at him, but he is reprimanded only with "you cannot force a woman to love you" rather than "you should never force a woman sexually, you jerk"... From a woman's point of view, the movie is a let down. It sort of "throws a bone" to women in letting them be both skilled fighters and leaders, but the movie is much more defined by the romance - which is characterized by the notion that human sexuality must involve an imbalance of power, with men dominating the woman the

The first step in processing the reviews is to make sure that any html tags that appear should be removed. In addition we wish to tokenize our input, that way words such as *entertained* and *entertaining* are considered the same with regard to sentiment analysis.

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

def review_to_words(review):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

The `review_to_words` method defined above uses `BeautifulSoup` to remove any html tags that appear and uses the `nltk` package to tokenize the reviews.  apply `review_to_words` to one of the reviews in the training set.

In [8]:
# Apply review_to_words to a review (train_X[100] or any other review)
review_to_words(train_X[100])

['believ',
 'none',
 'offici',
 'review',
 'movi',
 'warn',
 'peopl',
 'contain',
 'two',
 'quit',
 'upset',
 'sexual',
 'assault',
 'scene',
 'though',
 'cultur',
 'accept',
 'kind',
 'behavior',
 'simpli',
 'sexual',
 'violent',
 'biggest',
 'problem',
 'movi',
 'seem',
 'condemn',
 'assault',
 'woman',
 'repeatedli',
 'assault',
 'pressur',
 'never',
 'hold',
 'men',
 'account',
 'action',
 'neither',
 'anyon',
 'els',
 'one',
 'man',
 'stop',
 'complet',
 'assault',
 'someon',
 'throw',
 'dagger',
 'reprimand',
 'cannot',
 'forc',
 'woman',
 'love',
 'rather',
 'never',
 'forc',
 'woman',
 'sexual',
 'jerk',
 'woman',
 'point',
 'view',
 'movi',
 'let',
 'sort',
 'throw',
 'bone',
 'women',
 'let',
 'skill',
 'fighter',
 'leader',
 'movi',
 'much',
 'defin',
 'romanc',
 'character',
 'notion',
 'human',
 'sexual',
 'must',
 'involv',
 'imbal',
 'power',
 'men',
 'domin',
 'woman',
 'love',
 'amaz',
 'martial',
 'art',
 'fighter',
 'use',
 'fight',
 'skill',
 'tri',
 'fend',
 'attac

The method below applies the `review_to_words` method to each of the reviews in the training and testing datasets. In addition it caches the results. This is because performing this processing step can take a long time. This way if you are unable to complete the notebook in the current session, you can come back without needing to process the data a second time.

In [9]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        #words_train = list(map(review_to_words, data_train))
        #words_test = list(map(review_to_words, data_test))
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [10]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

Wrote preprocessed data to cache file: preprocessed_data.pkl


## Transform the data

Transformed the data from its word representation to a bag-of-words feature representation. To start, represent each word as an integer.

Using a recurrent neural network, it will be convenient if the length of each review is the same. To do this, I will fix a size for the reviews and then pad short reviews with the category 'no word' (which we will label `0`) and truncate long reviews.

To begin with, we need to construct a way to map words that appear in the reviews to integers by fixing the size of the vocabulary (including the 'no word' and 'infrequent' categories) to `5000`

In [13]:
import numpy as np

def build_dict(data, vocab_size = 5000):
    """Construct and return a dictionary mapping each of the most frequently appearing words to a unique integer."""
    
   
    word_count = {} # A dict storing the words that appear in the reviews along with how often they occur
    for sentence in data:
        for word in sentence:
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1
    
    
   
    sorted_words = sorted(word_count, key=word_count.get, reverse=True)
    
    word_dict = {} # This is what we are building, a dictionary that translates words into integers
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_dict

In [14]:
word_dict = build_dict(train_X)

In [15]:
# determine the five most frequently appearing words in the training set.
count = 0
for word, idx in word_dict.items():
    print(word)
    count += 1
    if count == 5:
        break;

movi
film
one
like
time


### Save `word_dict`

Later on when i construct an endpoint which processes a submitted review i will need to make use of the `word_dict` which I have created. As such, i will save it to a file now for future use.

In [16]:
data_dir = '../data/pytorch' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

In [17]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

### Transform the reviews

Convert reviews to their integer sequence representation, making sure to pad or truncate to a fixed length, which in this case is `500`.

In [18]:
def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [19]:
train_X, train_X_len = convert_and_pad_data(word_dict, train_X)
test_X, test_X_len = convert_and_pad_data(word_dict, test_X)

As a quick check to make sure that things are working as intended, check to see what one of the reviews in the training set looks like after having been processeed.

In [20]:
# Use this cell to examine one of the processed reviews to make sure everything is working as intended.
train_X[0]

array([   1,    2, 2196,  158,  321,  407,   51,   49,   12,  195, 1288,
          2,  187,    1,    3,  421,  335,   44,  368,   12, 1953,    1,
          2,    1,   12,  241,    1,    1, 1000,    1,    1,  349, 2406,
          1,  453,    2,  255,  207,    4,   53,    2,   56,   47,  184,
        195,   27,  184, 1715,  201,   28,  258, 1023,  162,  848,  761,
        163,  244,  118,  415,  131,   66,    1,   11,   62,  244,  163,
       3585,  258,  792, 1023,   23, 2062,    3,  163,  140,   46,   35,
       1586,  187,  353, 1345,   92,  429,   51,   99,    2,   36,  255,
         31,   30,    5,   41,  658,   32,  833,   97, 3049,   42, 2377,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

## Step 3: Upload the data to S3

Upload the training dataset to S3 in order for the training code to access it. For now let save it locally for upload to S3 later on.

### Save the processed training dataset locally

It is important to note the format of the data that we are saving as we will need to know it when we write the training code. In this case, each row of the dataset has the form `label`, `length`, `review[500]` where `review[500]` is a sequence of `500` integers representing the words in the review.

In [21]:
import pandas as pd
    
pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1) \
        .to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

### Uploading the training data


Next, upload the training data to the SageMaker default S3 bucket so that we can provide access to it while training our model.

In [22]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/sentiment_rnn'

role = sagemaker.get_execution_role()

In [23]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

**NOTE:** The cell above uploads the entire contents of our data directory. This includes the `word_dict.pkl` file. This is fortunate as we will need this later on when we create an endpoint that accepts an arbitrary review. For now, just take note of the fact that it resides in the data directory (and so also in the S3 training bucket) and that we will need to make sure it gets saved in the model directory.

## Step 4: Build and Train the PyTorch Model

In particular, a model comprises three objects

 - Model Artifacts,
 - Training Code, and
 - Inference Code,
 
each of which interact with one another. I am going to use containers provided by Amazon

Implementing the neural network in PyTorch along with a training script. I have provided the necessary model object in the `model.py` file, inside of the `train` folder. You can see the provided implementation by running the cell below.

In [24]:
!pygmentize train/model.py

import torch.nn as nn

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig

First I will load a small portion of the training data set to use as a sample. It would be very time consuming to try and train the model completely in the notebook as i do not have access to a gpu and the compute instance that am using is not particularly powerful. However, I can work on a small bit of the data.

In [25]:
import torch
import torch.utils.data

# Read in only the first 250 rows
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None, nrows=250)

# Turn the input pandas dataframe into tensors
train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

# Build the dataset
train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
# Build the dataloader
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=50)

### Writing the training method

Next is to write the training code. 

In [26]:
def train(model, train_loader, epochs, optimizer, loss_fn, device):
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for batch in train_loader:         
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            optimizer.zero_grad()
            
            # make predictions
            output = model(batch_X)
            
            # compute loss
            loss = loss_fn(output, batch_y)
            loss.backward()
            
            # update weights
            optimizer.step()
            
            total_loss += loss.data.item()
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_loader)))

#### Testing the train method

In [27]:
import torch.optim as optim
from train.model import LSTMClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(32, 100, 5000).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()

train(model, train_sample_dl, 5, optimizer, loss_fn, device)

Epoch: 1, BCELoss: 0.694963276386261
Epoch: 2, BCELoss: 0.6829926013946533
Epoch: 3, BCELoss: 0.6728263616561889
Epoch: 4, BCELoss: 0.6620816111564636
Epoch: 5, BCELoss: 0.6496333479881287


In order to construct a PyTorch model using SageMaker we must provide SageMaker with a training script. We may optionally include a directory which will be copied to the container and from which our training code will be run. When the training container is executed it will check the uploaded directory (if there is one) for a `requirements.txt` file and install any required Python libraries, after which the training script will be run.

### Training the model

Inside of the `train` directory is a file called `train.py` which contains most of the necessary code to train the model. The only thing that is missing is the implementation of the `train()` method which is writting above in this notebook.


In [35]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="train",
                    role=role,
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    hyperparameters={
                        'epochs': 10,
                        'hidden_dim': 200,
                    })

In [37]:
estimator.fit({'training': input_data})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-08-10 11:02:25 Starting - Starting the training job...
2020-08-10 11:02:27 Starting - Launching requested ML instances......
2020-08-10 11:03:32 Starting - Preparing the instances for training......
2020-08-10 11:04:54 Downloading - Downloading input data......
2020-08-10 11:05:36 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-08-10 11:06:01,025 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-08-10 11:06:01,049 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-08-10 11:06:04,072 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-08-10 11:06:04,349 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-08-10 11:06:04,350 sagemaker-containers INFO     Generating setup.cfg
2020-08-10 11:06:04,350 sagemaker-container

## Step 5: Testing the model

In this step, I will be testing this model by first deploying it and then sending the testing data to the deployed endpoint. I will do this to make sure that the deployed model is working correctly.

## Step 6: Deploy the model for testing

Currently the model takes input of the form `review_length, review[500]` where `review[500]` is a sequence of `500` integers which describe the words present in the review, encoded using `word_dict`.

There is need to provide a function which loads the saved model. This function must be called `model_fn()` from the `train.py` and takes as its only parameter a path to the directory where the model artifacts are stored. 

**NOTE**: When the built-in inference code is run it must import the `model_fn()` method from the `train.py` file. This is why the training code is wrapped in a main guard ( ie, `if __name__ == '__main__':` )

Since we don't need to change anything in the code that was uploaded during training, we can simply deploy the current model as-is.

In [38]:
# Deploy the trained model
predictor = estimator.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


---------------!

## Step 7 - Use the model for testing

Once deployed, read in the test data and send it off to the deployed model to get some results. This is to test the accuracy of our deployed model.

In [39]:
test_X = pd.concat([pd.DataFrame(test_X_len), pd.DataFrame(test_X)], axis=1)

In [40]:
# We split the data into chunks and send each chunk seperately, accumulating the results.

def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = np.array([])
    for array in split_array:
        predictions = np.append(predictions, predictor.predict(array))
    
    return predictions

In [41]:
predictions = predict(test_X.values)
predictions = [round(num) for num in predictions]

In [42]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.84124

###  More testing
Using the `review_to_words` and `convert_and_pad` methods created earlier, convert `test_review` into a numpy array `test_data` suitable to send to the model since our model expects input of the form `review_length, review[500]`.

In [54]:
test_review = 'The simplest pleasures in life are the best, and this film is one of them. Combining a rather basic storyline of love and adventure this movie transcends the usual weekend fair with wit and unmitigated charm.'

In [65]:
# Convert test_review into a form usable by the model and save the results in test_data
test_data = [review_to_words(test_review)]
test_data = convert_and_pad_data(word_dict, test_data)
test_data = pd.concat([pd.DataFrame(test_data[1]), pd.DataFrame(test_data[0])], axis=1)

Now that the review is processed, the resulting array can be sent to our model to predict the sentiment of the review.

In [66]:
predictor.predict(test_data)

array(0.8827809, dtype=float32)

Since the return value of our model is close to `1`, we can be certain that the review we submitted is positive.

### Delete the endpoint

Once an endpoint is deployed it continues to run until it is shut down. Since we are done using our endpoint for now, we can delete it.

In [67]:
estimator.delete_endpoint()

estimator.delete_endpoint() will be deprecated in SageMaker Python SDK v2. Please use the delete_endpoint() function on your predictor instead.


## Step 6 - Deploy the model for the web app


In [92]:
!pygmentize serve/predict.py

import argparse
import json
import os
import pickle
import sys
import sagemaker_containers
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

from model import LSTMClassifier

from utils import review_to_words, convert_and_pad

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LSTMClassifier(model_info['embedding_dim'], model_info['hidden_dim'], model_info['vocab_size'])

    # Load the store model parameters.
    model_path = os.path.join(mod

In [107]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='0.4.0',
                     entry_point='predict.py',
                     source_dir='serve',
                     predictor_cls=StringPredictor)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


---------------!

In [112]:
import glob
  # load the first `250` positive and negative reviews and send them to the endpoint

def test_reviews(data_dir='../data/aclImdb', stop=250):
    
    results = []
    ground = []
    
    # We make sure to test both positive and negative reviews    
    for sentiment in ['pos', 'neg']:
        
        path = os.path.join(data_dir, 'test', sentiment, '*.txt')
        files = glob.glob(path)
        
        files_read = 0
        
        print('Starting ', sentiment, ' files')
        
        # Iterate through the files and send them to the predictor
        for f in files:
            with open(f) as review:
                # First, we store the ground truth (was the review positive or negative)
                if sentiment == 'pos':
                    ground.append(1)
                else:
                    ground.append(0)
                # Read in the review and convert to 'utf-8' for transmission via HTTP
                review_input = review.read().encode('utf-8')
                # Send the review to the predictor and store the results
                results.append(float(predictor.predict(review_input)))
                
            # Sending reviews to our endpoint one at a time takes a while so we
            # only send a small number of reviews
            files_read += 1
            if files_read == stop:
                break
            
    return ground, results

In [113]:
ground, results = test_reviews()

Starting  pos  files
Starting  neg  files


In [114]:
from sklearn.metrics import accuracy_score
accuracy_score(ground, results)

0.852

As an additional test, send the `test_review` used for testing earlier.

In [115]:
predictor.predict(test_review)

b'1.0'

The endpoint is working as expected

In [116]:
predictor.endpoint

'sagemaker-pytorch-2020-08-11-13-46-03-919'

### Delete the endpoint


In [117]:
predictor.delete_endpoint()